In [1]:
import sys
sys.path.append('../')

import meta_dataloader.TCGA
import models.mlp
import numpy as np
import collections

%load_ext autoreload
%autoreload 2

In [2]:
tasks = meta_dataloader.TCGA.TCGAMeta(download=True)

In [3]:
task = tasks[30]
print(task.id)
taskdata = task[:]
print(taskdata[0].shape)
print(np.asarray(taskdata[1]).shape)
print(collections.Counter(taskdata[1]))

('Node_nature2012', 'BRCA')
(791, 16300)
(791,)
Counter({0: 385, 1: 264, 2: 99, 3: 43})


In [4]:
import sklearn.metrics

In [5]:
model = models.mlp.MLP(name="MLP_lay2_chan512", num_layer=2, channels=512, 
                       metric=sklearn.metrics.accuracy_score)


In [6]:
a = model.fit(taskdata[0], np.asarray(taskdata[1]))

epoch: 0, time: 0.13, valid_auc: 0.38, train_auc: 0.42
epoch: 1, time: 0.13, valid_auc: 0.48, train_auc: 0.49
epoch: 2, time: 0.12, valid_auc: 0.48, train_auc: 0.49
epoch: 3, time: 0.14, valid_auc: 0.48, train_auc: 0.49
epoch: 4, time: 0.13, valid_auc: 0.48, train_auc: 0.49
epoch: 5, time: 0.12, valid_auc: 0.48, train_auc: 0.49
epoch: 6, time: 0.13, valid_auc: 0.48, train_auc: 0.49
epoch: 7, time: 0.12, valid_auc: 0.48, train_auc: 0.49
epoch: 8, time: 0.12, valid_auc: 0.45, train_auc: 0.54
epoch: 9, time: 0.16, valid_auc: 0.45, train_auc: 0.55
epoch: 10, time: 0.24, valid_auc: 0.48, train_auc: 0.50
epoch: 11, time: 0.16, valid_auc: 0.46, train_auc: 0.55
epoch: 12, time: 0.16, valid_auc: 0.45, train_auc: 0.57
epoch: 13, time: 0.14, valid_auc: 0.44, train_auc: 0.57
epoch: 14, time: 0.19, valid_auc: 0.46, train_auc: 0.57
epoch: 15, time: 0.15, valid_auc: 0.44, train_auc: 0.57
total train time:15.61 for epochs: 16


In [33]:
a[1].shape

(632, 4)

In [43]:
np.argmax(a[1])

2184

In [44]:
sklearn.metrics.accuracy_score(a[0], np.argmax(a[1], axis=1))

0.4873417721518987